In [3]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession.builder \
        .master("spark://192.168.2.156:7077") \
        .appName("Zijie_V3_B") \
        .config("spark.dynamicAllocation.enabled", True) \
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
        .config("spark.shuffle.service.enabled", False) \
        .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
        .config("spark.executor.cores", 2) \
        .config("spark.driver.port", 9999) \
        .config("spark.blockManager.port", 10005) \
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

In [ ]:
spark_session.stop()

B.1

In [11]:
from pyspark.sql import SparkSession

# Load CSV file from HDFS
df = spark.read.csv("hdfs://192.168.2.156:9000/data/los-angeles-parking-citations/parking-citations.csv", header=True, inferSchema=True)

# Show the first few rows to verify loading
df.show()

# Count the number of rows in the DataFrame
row_count = df.count()
print("Number of rows in the dataset:", row_count)


+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    NULL|       NULL|            CA|         200304.0|NULL|HOND|        PA|   GY|

[Stage 39:====================================================>   (15 + 1) / 16]

Number of rows in the dataset: 13077724


B.2

In [12]:
# Print the schema of the DataFrame
df.printSchema()


root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



B.3

In [13]:
# Count the number of rows in the DataFrame
row_count = df.count()
print("Number of rows in the dataset:", row_count)


[Stage 42:====================================================>   (15 + 1) / 16]

Number of rows in the dataset: 13077724


B.4

In [14]:
# Get the number of partitions in the underlying RDD
num_partitions = df.rdd.getNumPartitions()
print("Number of partitions:", num_partitions)


Number of partitions: 16


B.5

In [17]:
# Drop the specified columns
df = df.drop("VIN", "Latitude", "Longitude")

# Show the updated DataFrame to verify
df.show()


[Stage 46:>                                                         (0 + 1) / 1]

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|Agency Description|Color Description|Body Style Description|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    NULL|       NULL|            CA|         200304.0|HOND|        PA|   GY|     13147 WELBY WAY|01521|   1.0|        4000A1|   NO EVIDENCE OF REG|       50

B.6

In [18]:
from pyspark.sql.functions import col, max

# Convert 'Fine amount' column to float (if it's not already)
df = df.withColumn("Fine amount", col("Fine amount").cast("float"))

# Find the maximum fine amount
max_fine = df.select(max("Fine amount")).collect()[0][0]
print("Maximum fine amount:", max_fine)

# Count the number of fines with this maximum amount
max_fine_count = df.filter(col("Fine amount") == max_fine).count()
print(f"Number of fines with the maximum amount ({max_fine}):", max_fine_count)


Maximum fine amount: 1100.0


25/03/10 12:57:00 ERROR TransportClient: Failed to send RPC RPC 8329072705657477534 to /192.168.2.237:50210: io.netty.channel.StacklessClosedChannelException
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source)
[Stage 50:=============================================>          (13 + 3) / 16]

Number of fines with the maximum amount (1100.0): 626


B.7

In [19]:
from pyspark.sql.functions import col, count

# Count occurrences of each vehicle make
top_vehicle_makes = df.groupBy("Make") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .limit(20)

# Show the top 20 most frequent vehicle makes
top_vehicle_makes.show(20, truncate=False)


+----+-------+
|Make|count  |
+----+-------+
|TOYT|2150768|
|HOND|1479996|
|FORD|1116235|
|NISS|945133 |
|CHEV|892676 |
|BMW |603092 |
|MERZ|543298 |
|VOLK|432030 |
|HYUN|404917 |
|DODG|391686 |
|LEXS|368420 |
|KIA |328155 |
|JEEP|316300 |
|AUDI|255395 |
|MAZD|242344 |
|OTHR|205546 |
|GMC |184889 |
|INFI|174315 |
|CHRY|159948 |
|SUBA|154640 |
+----+-------+



B.8

In [20]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define the mapping dictionary for colors
COLORS = { 
   'AL': 'Aluminum', 'AM': 'Amber', 'BG': 'Beige', 'BK': 'Black',  
   'BL': 'Blue', 'BN': 'Brown', 'BR': 'Brown', 'BZ': 'Bronze',  
   'CH': 'Charcoal', 'DK': 'Dark', 'GD': 'Gold', 'GO': 'Gold',  
   'GN': 'Green', 'GY': 'Gray', 'GT': 'Granite', 'IV': 'Ivory',  
   'LT': 'Light', 'OL': 'Olive', 'OR': 'Orange', 'MR': 'Maroon',  
   'PK': 'Pink', 'RD': 'Red', 'RE': 'Red', 'SI': 'Silver', 'SL': 'Silver',  
   'SM': 'Smoke', 'TN': 'Tan', 'VT': 'Violet', 'WT': 'White', 'WH': 'White',  
   'YL': 'Yellow', 'YE': 'Yellow', 'UN': 'Unknown' 
}

# Create a UDF to expand color abbreviations
def expand_color(color):
    return COLORS.get(color, color)  # Return mapped color or original if not found

# Register the UDF with Spark
expand_color_udf = udf(expand_color, StringType())

# Apply the UDF to create a new column 'color_long'
df = df.withColumn("color_long", expand_color_udf(df["Color"]))

# Show some sample data to verify the transformation
df.select("Color", "color_long").show(10, truncate=False)


[Stage 56:>                                                         (0 + 1) / 1]

+-----+----------+
|Color|color_long|
+-----+----------+
|GY   |Gray      |
|WH   |White     |
|BK   |Black     |
|WH   |White     |
|BK   |Black     |
|GY   |Gray      |
|BL   |Blue      |
|BK   |Black     |
|BR   |Brown     |
|SI   |Silver    |
+-----+----------+
only showing top 10 rows



B.9

In [21]:
from pyspark.sql.functions import count, desc

# Filter for Toyota (TOYT) vehicles
toyota_df = df.filter(df["Make"] == "TOYT")

# Count occurrences of each color
toyota_color_counts = toyota_df.groupBy("color_long") \
    .agg(count("*").alias("count")) \
    .orderBy(desc("count"))

# Show the most frequent color for Toyota vehicles
toyota_color_counts.show(1, truncate=False)


[Stage 57:====================================================>   (15 + 1) / 16]

+----------+------+
|color_long|count |
+----------+------+
|Gray      |489697|
+----------+------+
only showing top 1 row

